## Import Packages

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from scipy.stats import ttest_ind
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import proportions_chisquare

## Import Data

In [2]:
NYR=pd.read_csv("C:/Users/Repai/Downloads/New_years_resolutions.csv")

### Print First 5 Rows

In [3]:
NYR.head()

,tweet_created,tweet_text,tweet_category,tweet_topics,tweet_location,tweet_state,tweet_region,user_timezone,user_gender,retweet_count
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,"City of Angels, CA",CA,West,Pacific Time (US & Canada),male,NaN
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,"Arlington, TX",TX,South,Central Time (US & Canada),male,NaN
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,"Las Vegas, NV",NV,West,Central Time (US & Canada),male,NaN
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,"Ft. Luderdale, Florida",FL,South,NaN,male,NaN
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,South Carolina,SC,South,Pacific Time (US & Canada),female,NaN


## Wrangling

### Rename Columns

In [4]:
NYR.rename(columns={'tweet_created' : 'Date', 'tweet_text' : 'Tweet', 'tweet_category' : 'Category', 'tweet_topics' : 'Topics', 'tweet_location' : 'Location', 'tweet_state' : 'State', 'tweet_region' : 'Region', 'user_timezone' : 'TZ', 'user_gender' : 'Gender'}, inplace=True)

In [5]:
NYR.head()

,Date,Tweet,Category,Topics,Location,State,Region,TZ,Gender,retweet_count
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,"City of Angels, CA",CA,West,Pacific Time (US & Canada),male,NaN
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,"Arlington, TX",TX,South,Central Time (US & Canada),male,NaN
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,"Las Vegas, NV",NV,West,Central Time (US & Canada),male,NaN
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,"Ft. Luderdale, Florida",FL,South,NaN,male,NaN
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,South Carolina,SC,South,Pacific Time (US & Canada),female,NaN


### Drop Columns

In [6]:
NYRtrim = NYR.drop(['Location', 'retweet_count', 'TZ'], axis=1)

In [7]:
NYRtrim.head()

,Date,Tweet,Category,Topics,State,Region,Gender
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,CA,West,male
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,TX,South,male
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,NV,West,male
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,FL,South,male
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,SC,South,female


# Analysis

## Goodness of Fit Chi Square

In [8]:
NYRtrim.Gender.value_counts()

female    2367
male      2356
Name: Gender, dtype: int64

### Run the Analysis

In [9]:
observed_values_MF = scipy.array([2367, 2356])

C:\Users\Repai\AppData\Local\Temp\ipykernel_2312\3764035861.py:1: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  observed_values_MF = scipy.array([2367, 2356])


### Create Variable for Expected Values

#### 2367 + 2356 = 4723, 4723 * .9 = 4251 (90%), 4723 - 4251 = 472

In [10]:
expected_values_MF = scipy.array([4251, 472])

C:\Users\Repai\AppData\Local\Temp\ipykernel_2312\3954925786.py:1: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  expected_values_MF = scipy.array([4251, 472])


In [11]:
scipy.stats.chisquare(observed_values_MF, f_exp=expected_values_MF)

Power_divergenceResult(statistic=8355.003552504097, pvalue=0.0)

## Independent t-test

## Wrangling

### Recoding

In [12]:
def recode (series):
    if series == "female":
        return 0
    if series == "male":
        return 1
NYRtrim['Genderr'] = NYRtrim['Gender'].apply(recode)

In [13]:
def recode (series):
    if series == "Humor":
        return 1
    if series == "Health & Fitness":
        return 2
    if series == "Personal Growth":
        return 3
    if series == "Finance":
        return 4
    if series == "Philanthropic":
        return 5
    if series == "Recreation & Leisure":
        return 6
    if series == "Time Management/Organization":
        return 7
    if series == "Family/Friends/Relationships":
        return 8
    if series == "Career":
        return 9
    if series == "Education/Training":
        return 10
NYRtrim['Categoryr'] = NYRtrim['Category'].apply(recode)

In [34]:
NYRtrim.head()

,Date,Tweet,Category,Topics,State,Region,Gender,Genderr,Categoryr
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,CA,West,male,1,1
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,TX,South,male,1,2
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,NV,West,male,1,3
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,FL,South,male,1,1
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,SC,South,female,0,3


### Run the Analysis

In [40]:
ttest_ind(NYRtrim["Categoryr"], NYRtrim["Genderr"])

Ttest_indResult(statistic=86.45681341155864, pvalue=0.0)